### Problem Statement:

Consider we have a table of food items and their respective nutrient & price information. You are required to maintain a diet with the minimum requirements of each nutrient. How much of each food would you include in a meal to comply with your diet limitations while minimizing your cost?

In our problem, we have 21 food items and 6 nutrients. Use DP_input_xxx.csv files.


In [1]:
import pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

In [2]:
DP_input_food_nutrients_df = pd.read_csv('data/DP_input_food_nutrients.csv')
DP_input_min_intake_df = pd.read_csv('data/DP_input_min_intake.csv')
DP_input_food_df = pd.read_csv('data/DP_input_food.csv')

In [4]:
DP_input_food_nutrients_df

,Food,Nutr,Value
0,Corn meal,Calories,360
1,Corn meal,Protein,9
2,Corn meal,Fat,4
3,Corn meal,Sat.Fat,2
4,Corn meal,Fiber,1
...,...,...,...
121,Peanut butter,Protein,12
122,Peanut butter,Fat,25
123,Peanut butter,Sat.Fat,17
124,Peanut butter,Fiber,0


In [5]:
DP_input_min_intake_df

,Nutr,Req
0,Calories,1800
1,Protein,45
2,Fat,40
3,Sat.Fat,0
4,Fiber,26
5,Carbs,202


In [6]:
DP_input_food_df

,Food,Cost
0,Beef,6.00
1,Cheese,0.50
2,Corn meal,0.50
3,Cornflakes,0.50
4,Cows milk,3.28
5,Crackers,0.50
6,Cream cheese,0.30
7,Eggs raw,0.40
8,Flounder,5.00
9,Flour,1.00


### Layman Formulation:

##### DECISION VARIABLE: 
Pounds of each food item

##### OBJECTIVE FUNCTION: 
Minimizing the total cost

##### CONSTRAINTS: 
Satisfy Nutrient Requirements


### Compact Mathematical Formulation:

##### SETS:
FOODS = (FoodA, FoodB, FoodC, ..)
NUTRS = (Nutr1, Nutr2, ..)

##### DECISION VARIABLE: 
PoundsF for all F in FOODS

##### PARAMETERS:
1. contFN - Content of Nutrient N in Food F for all F & N
2. costF - Cost of Food F for all F in FOODS
3. reqN - Requirement of each Nutrient N for all N in NUTRS

##### OBJECTIVE FUNCTION: 
min (sum over F (costF * PoundsF))

##### CONSTRAINTS: 
1. sum over F (contFN * PoundsF) >= reqN for each N; 
2. PoundsF >= 0 for all F in FOODS

In [7]:
m = AbstractModel()

#Sets
m.FOODS = Set()
m.NUTR = Set()

#Vars
m.Pounds = Var(m.FOODS, domain = NonNegativeReals)

#Params
m.cost = Param(m.FOODS)
m.req = Param(m.NUTR)
m.cont = Param(m.FOODS, m.NUTR)

#Constraints
def min_content_Rule(m, n):
    return sum(m.cont[f,n] * m.Pounds[f] for f in m.FOODS) >= m.req[n]
m.min_content = Constraint(m.NUTR, rule = min_content_Rule)

def total_Cost_Rule(m):
    return sum(m.cost[f] * m.Pounds[f] for f in m.FOODS)
m.total_Cost = Objective(rule = total_Cost_Rule, sense = minimize)

In [8]:
instanceData = {None:{
    'FOODS': set(DP_input_food_nutrients_df['Food']),
    'NUTR': DP_input_min_intake_df['Nutr'],
    'cost': DP_input_food_df.set_index(['Food']).to_dict()['Cost'],
    'req': DP_input_min_intake_df.set_index(['Nutr']).to_dict()['Req'],
    'cont': DP_input_food_nutrients_df.set_index(['Food', 'Nutr']).to_dict()['Value']
}}

# Build instance
instance = m.create_instance(instanceData)

source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo


In [10]:
solver = SolverFactory('glpk')
#Solve
sol = solver.solve(instance)
print(sol['Solver'])


- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.040268898010253906



In [16]:
print("The total cost is " + str(instance.total_Cost()))
print("The food items and their respective weight required in pounds - ")
for f in instance.FOODS:
    if instance.Pounds[f]() > 0:
        print(f + " of " + str(instance.Pounds[f]()) + " pounds")

The total cost is 3.967211055276381
The food items and their respective weight required in pounds - 
Noodles of 3.4786432160804 pounds
Oatmeal of 6.5 pounds
Cream cheese of 1.23115577889447 pounds
